# Delete this as soon as you see it and don't know why its here.

In [14]:
!pip install openai

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [25]:
import json

with open("representative_questions.json", "r", encoding="utf-8") as f:
    representative_questions = json.load(f)


with open("edital_name_to_file_path.json", "r",  encoding="utf-8") as f:
    edital_name_to_file_path = json.load(f)["editais"]

from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

vector_to_edital = []
for edital in representative_questions["editais"].keys():
    for question in representative_questions["editais"][edital]:
        vector = embeddings.embed_query(question)
        vector_to_edital.append((vector, edital))

In [32]:
from scipy.spatial.distance import cosine

def find_edital_by_question(question):
    question_vector = embeddings.embed_query(question)
    closest_edital = None
    similarities = []
    # biggest_similarity = -1  # cosine similarity ranges [-1, 1]

    for vector, edital in vector_to_edital:
        cos_sim = 1 - cosine(question_vector, vector)
        # if cos_sim > biggest_similarity:
        #     biggest_similarity = cos_sim
        #     closest_edital = edital
        similarities.append((cos_sim, edital))
    
    similarities.sort(reverse=True)
    editais_count = {}
    for similiraty in similarities[0:5]:
        edital = similiraty[1]
        if edital in editais_count.keys():
            editais_count[edital] += 1
        else:
            editais_count[edital] = 1

    max_count = -1
    max_edital = None
    for edital, value in editais_count.items():
        if value > max_count:
            max_count = value
            max_edital = edital

    return max_edital

def get_edital_file_content(edital_name):
    """
    Given the name of an edital, returns its content as a string.
    """
    file_path = edital_name_to_file_path[edital_name]
    with open(file_path, "r", encoding="utf-8") as file:
        return file.read()
    

find_edital_by_question("Quais requisitos devo atender para participar do cadastro socioeconômico?")


'Edital Cadastro Socioeconômico (UFCG/PRAC/CGAE Nº 03/2025)'

In [34]:
import importlib
import prompt
import call_model_dell

# Reload modules
importlib.reload(prompt)
importlib.reload(call_model_dell)

# Re-import functions to get the latest definitions
from prompt import prompt
from call_model_dell import call_model

def answer_question(question):
    edital_name = find_edital_by_question(question)
    edital_content = get_edital_file_content(edital_name)

    full_prompt = prompt(edital_content, question)

    answer = call_model(full_prompt)

    return answer, edital_name

x = answer_question("Quais requisitos devo atender para participar do cadastro socioeconômico?")

def print_with_line_breaks(text, n=10):
    words = text.split()
    for i in range(0, len(words), n):
        print(' '.join(words[i:i+n]))

print("Edital:", x[1])
print_with_line_breaks(x[0], 10)

Edital: Edital Cadastro Socioeconômico (UFCG/PRAC/CGAE Nº 03/2025)
Resposta = Para participar do processo de cadastramento socioeconômico, o/a
estudante deve atender cumulativamente aos seguintes requisitos: a. Estar regularmente
matriculado o u cadastrado para ingresso em cursos presenciais de
graduação da UFCG. b. Possuir grupo familiar com renda per
capita(por pessoa) menor ou igual a 1 (um) salário- mínimo,
entendendo-se por renda mensal per capita o valor bruto dos
rendimentos da família, dividido pelo número de dependentes; c. Atender
aos critéri


In [23]:
x

(' \nResposta = Não consigo responder a essa pergunta com minha base de informações.',
 'AUXÍLIO AATIVIDADES OBRIGATÓRIAS EXTERNAS')